# Import packages

In [16]:
import numpy as np
import time
from copy import deepcopy

# PQR

In [17]:
def PQR(matrix, n, m, r = 0, eps = 0):

    if not r or r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))
    P = np.arange(m)

    tol = eps + 1
    i = 0

    while i < r and tol > eps:

        norms = np.array([np.linalg.norm(matrix[:, j]) for j in range(i, m)])
        tol = np.sqrt(np.sum(norms**2))
        args = np.argsort(norms)[::-1]
        args += i

        P[[i, args[0]]] = P[[args[0], i]]
        R[:, [i, args[0]]]  = R[:, [args[0], i]]
        matrix[:, [i, args[0]]] = matrix[:, [args[0], i]]

        R[i, i] = norms[args[0] - i]
        Q[:, i] = matrix[:, i] / R[i, i]

        for j in range(i + 1, m):
            R[i, j] = matrix[:, j] @ Q[:, i]
            matrix[:, j] -= R[i, j] * Q[:, i]

        i += 1

    return Q, R, P, i

# MaxVol

In [18]:
def maxvol(mat, n, m, r, eps = 0):

    if not r or r > min(n, m):
          r = min(n, m)

    rows = np.random.permutation(n)
    B = mat[rows[:r], :]
    _, _, cols, rank = PQR(B, r, m, r, eps)


    rev_rows = np.zeros(n, dtype=int)
    rev_cols = np.zeros(m, dtype=int)

    for i in range(n):
        rev_rows[rows[i]] = i
    for i in range(m):
        rev_cols[cols[i]] = i

    matrix = mat[np.ix_(rows, cols)]

    R = np.arange(r)

    C = matrix[:, :r]
    A = C[:r, :]

    inv_A = np.linalg.inv(A)

    prod = C @ inv_A

    max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    iter = 0

    while np.abs(prod[max_val_ind]) > 1 and max_val_ind[0] >= r and iter < r * r:

        iter += 1

        vec = prod[max_val_ind[0], :].copy()
        vec[max_val_ind[1]] -= 1
        vec /= prod[max_val_ind]

        prod -= prod[:, max_val_ind[1]].reshape((-1, 1)) @ vec.reshape((1, -1))

        R[max_val_ind[1]] = max_val_ind[0]

        max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    prod = prod[rev_rows, :]
    R = matrix[np.ix_(R, rev_cols)]

    return prod, R

# Cross Approximation

In [19]:
def cross_approx(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.arange(m)

    i = 0

    max_el = eps + 1

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        ind = np.random.randint(m - i)
        col = vec[ind]

        row = np.argmax(np.abs(matrix[:, col]), axis=None)
        col = np.argmax(np.abs(matrix[row, :]), axis=None)
        row = np.argmax(np.abs(matrix[:, col]), axis=None)

        Q[:, i] = matrix[:, col] / matrix[row, col]
        R[i, :] = matrix[row, :]

        max_el = np.abs(matrix[row, col])

        matrix -= Q[:, i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

        ind = np.where(vec == col)[0][0]

        vec = np.delete(vec, ind)

        i += 1

    return Q, R, i

# Cross Approximation Dzheltkov

In [25]:
def cross_approx_dzh(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.arange(m)

    i = 0

    max_el = eps + 1

    rows = np.zeros((r, m))
    id_rows = np.ones(r, dtype=int) * n
    free_row = 0
    cols = np.zeros((n, r))
    id_cols = np.ones(r, dtype=int) * m
    free_col = 0

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        ind = np.random.randint(m - i)
        col = vec[ind]

        if col in id_cols:
            val = np.where(id_cols == col)[0][0]
            row = np.argmax(np.abs(cols[:, val]), axis=None)
        else:
            row = np.argmax(np.abs(matrix[:, col]), axis=None)
            id_cols[free_col] = col
            cols[:, free_col] = matrix[:, col]
            free_col += 1

        if row in id_rows:
            val = np.where(id_rows == row)[0][0]
            col = np.argmax(np.abs(rows[val, :]), axis=None)
        else:
            col = np.argmax(np.abs(matrix[row, :]), axis=None)
            id_rows[free_row] = row
            rows[free_row, :] = matrix[row, :]
            free_row += 1

        if col in id_cols:
            val = np.where(id_cols == col)[0][0]
            row = np.argmax(np.abs(cols[:, val]), axis=None)
        else:
            row = np.argmax(np.abs(matrix[:, col]), axis=None)

        if row in id_rows:
            val = np.where(id_rows == row)[0][0]
            rows = np.delete(rows, val, 0)
            id_rows = np.delete(id_rows, val)
            free_row -= 1

        if col in id_cols:
            val = np.where(id_cols == col)[0][0]
            cols = np.delete(cols, val, 1)
            id_cols = np.delete(id_cols, val)
            free_col -= 1


        Q[:, i] = matrix[:, col] / matrix[row, col]
        R[i, :] = matrix[row, :]

        matrix -= Q[:, i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

        if free_row:
            rows[:free_row, :] -= Q[id_rows[:free_row], i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))
        if free_col:
            cols[:, :free_col] -= Q[:, i].reshape((-1, 1)) @ R[i, id_cols[:free_col]].reshape((1, -1))

        ind = np.where(vec == col)[0][0]

        vec = np.delete(vec, ind)

        i += 1

        max_el = np.sqrt(np.linalg.norm(rows[:free_row, :])**2 + np.linalg.norm(cols[:, :free_col])**2 - np.linalg.norm(cols[id_rows[:free_row], :free_col])**2)

    return Q, R, i

# Create matrix

In [26]:
n = 10
m = 10
mat_cr = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        mat_cr[i, j] = 1 / (i + j + 1)

In [27]:
matrix = mat_cr.copy()
mat_copy = mat_cr.copy()

# Run Cross Approximation

In [28]:
start_time = time.time()
Q, R, rank = cross_approx_dzh(matrix, n, m, 7)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.01016545295715332
1.6228970729719229e-09


# Run PQR

In [104]:
start_time = time.time()
Q, R, P, rank = PQR(matrix, n, m, 10)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy[:, P]) / np.linalg.norm(mat_copy))

time = 0.0022275447845458984
6.9843349160528e-17


# Run MaxVol

In [53]:
start_time = time.time()
Q, R = maxvol(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.6902766227722168
1.296308361851803e-16


In [45]:
a = np.arange(12).reshape(3, 4)
print(a)
print()
a = np.delete(a, 2, 1)
print(a)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]

[[ 0  1  3]
 [ 4  5  7]
 [ 8  9 11]]


In [134]:
print(np.random.randint(10))

2
